# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark SQL: Soluciones de almacenamiento para Big Data (Couchbase)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [22]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [23]:
from pyspark.sql import SparkSession

# Hint for homework: .config("spark.jars.packages", "groupId:artifactId:version") \

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Couchbase-Write") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars.packages", "com.couchbase.client:spark-connector_2.13:3.5.2") \
    .config("spark.ui.port","4040") \
    .config("spark.couchbase.connectionString", "couchbases://couchbase-iteso") \
    .config("spark.couchbase.username", "Administrator") \
    .config("spark.couchbase.password", "admin1234") \
    .config("spark.ssl.insecure", "true") \
    .getOrCreate()
sc = spark.sparkContext

#### Cargamos el DF de Netflix

In [24]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|9/25/2021 |2020        |PG-13 |90 min  |Documentaries                                                |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |9/24/2021 |2021        |TV-MA |1 Season|Crime TV Shows, International TV Shows, TV Action & Adventure|
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|9/24/

#### Escribir un DF a Couchbase 

In [25]:
netflix_df = netflix_df.limit(5)

netflix_df.write \
    .format("couchbase.kv") \
    .option("bucket", "test-bucket") \
    .option("scope", "test-scope") \
    .option("collection", "test-collection") \
    .option("idFieldName", "show_id") \
    .mode("overwrite") \
    .save()

print("Done")

Done


In [26]:
# Stop the SparkContext
sc.stop()

#### Leer info desde Couchbase 

In [27]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Couchbase-Read") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars.packages", "com.couchbase.client:spark-connector_2.13:3.5.2") \
    .config("spark.ui.port","4040") \
    .config("spark.couchbase.connectionString", "couchbases://couchbase-iteso") \
    .config("spark.couchbase.username", "Administrator") \
    .config("spark.couchbase.password", "admin1234") \
    .config("spark.ssl.insecure", "true") \
    .getOrCreate()
sc = spark.sparkContext

##### Propiedades de conexión con la Base de Datos remota

In [28]:
df = spark.read.format("couchbase.query") \
    .option("bucket", "test-bucket") \
    .option("scope", "test-scope") \
    .option("collection", "test-collection") \
    .load()
df.printSchema()
df.show()

root
 |-- __META_ID: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- director: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)



+---------+-------------+----------+---------------+--------+--------------------+------+------------+--------------------+-------+
|__META_ID|      country|date_added|       director|duration|           listed_in|rating|release_year|               title|   type|
+---------+-------------+----------+---------------+--------+--------------------+------+------------+--------------------+-------+
|       s6|United States| 9/24/2021|  Mike Flanagan|1 Season|TV Dramas, TV Hor...| TV-MA|        2021|       Midnight Mass|TV Show|
|      s14|       Brazil| 9/22/2021|  Bruno Garotti|  91 min|Children & Family...| TV-PG|        2021|Confessions of an...|  Movie|
|       s1|United States| 9/25/2021|Kirsten Johnson|  90 min|       Documentaries| PG-13|        2020|Dick Johnson Is Dead|  Movie|
|       s8|United States| 9/24/2021|   Haile Gerima| 125 min|Dramas, Independe...| TV-MA|        1993|             Sankofa|  Movie|
|       s3|       France| 9/24/2021|Julien Leclercq|1 Season|Crime TV Shows,

In [29]:
sc.stop()